In [ ]:
import torch
model_path = ""
trained_state_dict = torch.load("ECGNet_8bit_trained_model.pth)["models_state_dict"][0]
trained_state_dict = torch.load("ECGNet_8bit_trained_model.pth")["state_dict"]

print(trained_state_dict)

In [ ]:
from copy import deepcopy

modified_model = deepcopy(model)

# print(modified_model.features[0].weight.data.detach().numpy())
W_orig = modified_model.features[1].weight.data.detach().numpy()
W_orig.shape

In [ ]:
from brevitas.core.quant import QuantType
from brevitas.nn import QuantIdentity


class BrevitasIEEEForExport(nn.Module):
    def __init__(self, my_pretrained_model):
        super(BrevitasIEEEForExport, self).__init__()
#         self.qnt_input = QuantIdentity(quant_type=QuantType.FP, bit_width=32)
        self.pretrained = my_pretrained_model
        self.qnt_output = QuantIdentity(quant_type=QuantType.INT, bit_width=8)
    
    def forward(self, x):
        # assume x contains bipolar {-1,1} elems
        # shift from {-1,1} -> {0,1} since that is the
        # input range for the trained network
#         x = (x + torch.tensor([1.0])) / 2.0  
        out_original = self.pretrained(x)
        out_final = self.qnt_output(out_original)   # output as {-1,1}     
        return out_final

model_for_export = BrevitasIEEEForExport(model)

In [ ]:
# import brevitas.onnx as bo
# from brevitas.quant_tensor import QuantTensor

# ready_model_filename = "brevitas07-IEEE-ready.onnx"
# input_shape = (1,1,1,128)
# # create a QuantTensor instance to mark input as bipolar during export
# input_a = np.random.randint(0, 1, size=input_shape).astype(np.float32)
# # input_a = 2 * input_a - 1
# scale = 1.0
# input_t = torch.from_numpy(input_a * scale)
# input_qt = QuantTensor(
#     input_t, scale=torch.tensor(scale), bit_width=torch.tensor(32.0), signed=True
# )

# bo.export_finn_onnx(
#     model, export_path=ready_model_filename, input_t=input_qt
# )

# print("Model saved to %s" % ready_model_filename)
import brevitas.onnx as bo
from brevitas.quant_tensor import QuantTensor

ready_model_filename = "brevitas09-IEEE-ready.onnx"
input_shape = (1,1,1,128)
# create a QuantTensor instance to mark input as bipolar during export
input_a = np.random.randint(0, 1, size=input_shape).astype(np.float32)
# input_a = 2 * input_a - 1
scale = 1.0

input_t = torch.from_numpy(input_a * scale)
input_qt = QuantTensor(
    input_t, scale=torch.tensor(scale), bit_width=torch.tensor(8.0), signed=True
)

# print(input_qt)
bo.export_finn_onnx(
    model_for_export, export_path=ready_model_filename, input_t=input_qt
)

print("Model saved to %s" % ready_model_filename)

In [ ]:
from finn.util.visualization import showInNetron

showInNetron(ready_model_filename)

In [ ]:
pred = model(input_t)
print(pred)

In [ ]:
from finn.core.modelwrapper import ModelWrapper

ready_model_filename = "brevitas09-IEEE-ready.onnx"
model_for_sim = ModelWrapper(ready_model_filename)
# dir(model_for_sim)

In [ ]:
from finn.core.datatype import DataType

finnonnx_in_tensor_name = model_for_sim.graph.input[0].name
finnonnx_out_tensor_name = model_for_sim.graph.output[0].name
print("Input tensor name: %s" % finnonnx_in_tensor_name)
print("Output tensor name: %s" % finnonnx_out_tensor_name)
finnonnx_model_in_shape = model_for_sim.get_tensor_shape(finnonnx_in_tensor_name)
finnonnx_model_out_shape = model_for_sim.get_tensor_shape(finnonnx_out_tensor_name)
print("Input tensor shape: %s" % str(finnonnx_model_in_shape))
print("Output tensor shape: %s" % str(finnonnx_model_out_shape))
finnonnx_model_in_dt = model_for_sim.get_tensor_datatype(finnonnx_in_tensor_name)
finnonnx_model_out_dt = model_for_sim.get_tensor_datatype(finnonnx_out_tensor_name)
print("Input tensor datatype: %s" % str(finnonnx_model_in_dt.name))
print("Output tensor datatype: %s" % str(finnonnx_model_out_dt.name))
print("List of node operator types in the graph: ")
print([x.op_type for x in model_for_sim.graph.node])

In [ ]:
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors

model_for_sim = model_for_sim.transform(InferShapes())
model_for_sim = model_for_sim.transform(FoldConstants())
model_for_sim = model_for_sim.transform(GiveUniqueNodeNames())
model_for_sim = model_for_sim.transform(GiveReadableTensorNames())
model_for_sim = model_for_sim.transform(InferDataTypes())
model_for_sim = model_for_sim.transform(RemoveStaticGraphInputs())
# model_for_sim = model_for_sim.transform(Change3DTo4DTensors())
model_for_sim = model_for_sim.transform(Change3DTo4DTensors())

verif_model_filename = "brevitas09-IEEE-verification.onnx"
model_for_sim.save(verif_model_filename)

In [ ]:
from finn.util.visualization import showInNetron

showInNetron(verif_model_filename)

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "brevitas09-IEEE-ready.onnx"

estimates_output_dir = "output_estimates_only"

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 10000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    board               = "Pynq-Z2",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

In [ ]:
! cat {estimates_output_dir}/report/estimate_network_performance.json

In [ ]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [ ]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

In [ ]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "brevitas09-IEEE-ready.onnx"

final_output_dir = "output_final"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")


cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 10000,
    synth_clk_period_ns = 10.0,
    board               = "Pynq-Z2",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg)